In [ ]:
%pip install pyhomogeneity

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyhomogeneity import pettitt_test, snht_test, buishand_u_test

In [ ]:
PATH_INPUT = "../data/raw/estaciones"
PATH_OUTPUT = "../data/curated"

In [ ]:
df_temp_LP = pd.read_excel(f"{PATH_INPUT}/temp-la-plata-aero.xlsx", skiprows=3, names=["fecha", "t_max", "t_min", "t_media"])
df_pp_LP = pd.read_excel(f"{PATH_INPUT}/pp-la-plata-aero.xlsx", skiprows=3, names=["fecha", "pp"])
df_punta_indio = pd.read_excel(f"{PATH_INPUT}/PUNTA_INDIO_B.A..xlsx", skiprows=5, names=["fecha", "hora", "temp", "pp"])
df_ezeiza = pd.read_excel(f"{PATH_INPUT}/EZEIZA_AERO.xlsx", skiprows=5, names=["fecha", "hora", "temp", "pp"])
df_aeroparque = pd.read_excel(f"{PATH_INPUT}/AEROPARQUE_AERO.xlsx", skiprows=5, names=["fecha", "hora", "temp", "pp"])
df_bsas_obs = pd.read_excel(f"{PATH_INPUT}/BUENOS_AIRES_OBSERVATORIO.xlsx", skiprows=5, names=["fecha", "hora", "temp", "pp"])

Primero, hacemos un merge de cada dataset, de forma de tener todo en un mismo dataset.

In [5]:
df_lp = pd.merge(df_temp_LP, df_pp_LP, on="fecha")

In [ ]:
def filtrar_fechas(df):
  df["fecha"] = pd.to_datetime(df["fecha"])
  return df[(df["fecha"].dt.year >= 1961) & (df["fecha"].dt.year <= 2024)]

df_lp = filtrar_fechas(df_lp)
df_punta_indio = filtrar_fechas(df_punta_indio)
df_ezeiza = filtrar_fechas(df_ezeiza)
df_aeroparque = filtrar_fechas(df_aeroparque)
df_bsas_obs = filtrar_fechas(df_bsas_obs)

Ahora verificamos los tipos de los datos y los convertimos a los correspondientes. Por ejemplo, para los datos que no son del tipo fecha, los convertimos a tipo numerico (float en este caso).

In [6]:
def to_numeric(df, col):
    df[col] = pd.to_numeric(df[col], errors="coerce")

df_lp["fecha"] = pd.to_datetime(df_lp["fecha"])
df_lp = df_lp[['fecha', 'pp', 't_min', 't_max', 't_media']]

for col in df_lp.columns[1:]:
    to_numeric(df_lp, col)

In [ ]:
def formatear_estaciones(df):
  df_mean = df.groupby("fecha", as_index=False).mean().rename(columns={"temp": "t_media"}).drop(columns=["hora"])
  df_max = df.groupby("fecha", as_index=False).max().rename(columns={"temp": "t_max"}).drop(columns=["pp", "hora"])
  df_min = df.groupby("fecha", as_index=False).min().rename(columns={"temp": "t_min"}).drop(columns=["pp", "hora"])

  return pd.merge(df_mean, df_min, on="fecha").merge(df_max, on="fecha")

df_aeroparque = formatear_estaciones(df_aeroparque)
df_bsas_obs = formatear_estaciones(df_bsas_obs)
df_ezeiza = formatear_estaciones(df_ezeiza)
df_punta_indio = formatear_estaciones(df_punta_indio)

In [ ]:
df_aeroparque

,fecha,t_media,pp,t_min,t_max
0,1961-01-01,28.225000,NaN,24.6,32.0
1,1961-01-02,24.400000,NaN,19.8,30.8
2,1961-01-03,23.175000,NaN,20.0,25.7
3,1961-01-04,24.600000,NaN,20.0,29.0
4,1961-01-05,22.750000,NaN,20.6,24.0
...,...,...,...,...,...
23370,2024-12-27,21.525000,NaN,17.4,24.8
23371,2024-12-28,21.241667,NaN,18.3,23.9
23372,2024-12-29,21.779167,NaN,19.5,25.2
23373,2024-12-30,23.083333,NaN,20.4,26.3


### Control de estructura y consistencia

Pasos a seguir:

- Verificación de que los datos esten dentro de un rango de valores que tenga sentido

- Verificación de que no exista "valores imposibles" tales como pp negativas, temperaturas irreales, t_media que no este entre la t_min y t_max.

In [ ]:
def control_calidad_inicial(df,
                            nombre_estacion="",
                            check_tmedia=True,
                            verbose=True):
    df = df.copy()

    if verbose:
        print("\n====================")
        print(f"CONTROL DE CALIDAD - {nombre_estacion}")
        print("====================\n")

    # 1. Duplicados
    duplicados = df['fecha'].duplicated().sum()
    if verbose:
        print(f"Fechas duplicadas: {duplicados}")

    # 2. Fechas faltantes
    fecha_min = df['fecha'].min()
    fecha_max = df['fecha'].max()
    fechas_completas = pd.date_range(fecha_min, fecha_max)
    faltantes = fechas_completas.difference(df['fecha'])

    if verbose:
        print(f"\nFechas faltantes detectadas: {len(faltantes)}")

    # 3. Rango físico
    errores_temp = df[
        (df['t_min'] < -50) | (df['t_min'] > 55) |
        (df['t_max'] < -50) | (df['t_max'] > 55)
    ]

    errores_pp = df[df['pp'] < 0] if 'pp' in df.columns else pd.DataFrame()

    if verbose:
        print(f"\nTemperaturas fuera de rango físico: {len(errores_temp)}")
        print(f"Precipitaciones negativas: {len(errores_pp)}")

    # 4. Coherencia t_min ≤ t_max
    incoherencias_minmax = df[df['t_min'] > df['t_max']]
    if verbose:
        print(f"\nt_min > t_max: {len(incoherencias_minmax)}")

    # 6. Saltos bruscos
    df_sorted = df.sort_values('fecha').set_index('fecha')
    saltos = {}

    for col in ['t_min', 't_max']:
        jumps = df_sorted[col].diff().abs() > 15
        saltos[col] = df_sorted[jumps]

        if verbose:
            print(f"\nSaltos bruscos en {col}: {len(saltos[col])}")

    return {
        "duplicados": duplicados,
        "fechas_faltantes": list(faltantes),
        "errores_temp": errores_temp,
        "errores_pp": errores_pp,
        "incoherencias_minmax": incoherencias_minmax,
        "saltos": saltos
    }

In [ ]:
qc_lp = control_calidad_inicial(
    df_lp,
    nombre_estacion="La Plata AERO",
    check_tmedia=True
)

qc_eze = control_calidad_inicial(
    df_ezeiza,
    nombre_estacion="Ezeiza AERO",
    check_tmedia=False
)

qc_aep = control_calidad_inicial(
    df_aeroparque,
    nombre_estacion="Aeroparque AERO",
    check_tmedia=False
)

qc_bsas = control_calidad_inicial(
    df_bsas_obs,
    nombre_estacion="Observatorio Bs.As.",
    check_tmedia=False
)

qc_punta = control_calidad_inicial(
    df_punta_indio,
    nombre_estacion="Punta Indio",
    check_tmedia=False
)



CONTROL DE CALIDAD - La Plata AERO

Fechas duplicadas: 0

Fechas faltantes detectadas: 0

Temperaturas fuera de rango físico: 0
Precipitaciones negativas: 0

t_min > t_max: 0

Saltos bruscos en t_min: 0

Saltos bruscos en t_max: 1

CONTROL DE CALIDAD - Ezeiza AERO

Fechas duplicadas: 0

Fechas faltantes detectadas: 119

Temperaturas fuera de rango físico: 0
Precipitaciones negativas: 0

t_min > t_max: 0

Saltos bruscos en t_min: 0

Saltos bruscos en t_max: 10

CONTROL DE CALIDAD - Aeroparque AERO

Fechas duplicadas: 0

Fechas faltantes detectadas: 1

Temperaturas fuera de rango físico: 0
Precipitaciones negativas: 0

t_min > t_max: 0

Saltos bruscos en t_min: 0

Saltos bruscos en t_max: 2

CONTROL DE CALIDAD - Observatorio Bs.As.

Fechas duplicadas: 0

Fechas faltantes detectadas: 3

Temperaturas fuera de rango físico: 0
Precipitaciones negativas: 0

t_min > t_max: 0

Saltos bruscos en t_min: 0

Saltos bruscos en t_max: 7

CONTROL DE CALIDAD - Punta Indio

Fechas duplicadas: 0

Fechas

In [ ]:
qc_lp["saltos"]["t_max"].index.tolist()

[Timestamp('2023-08-03 00:00:00')]

In [ ]:
def get_serie(df, col):
  return pd.Series(df[col].values, index=df['fecha'])

series_tmax = {
    "lp": get_serie(df_lp, "t_max"),
    "ezeiza": get_serie(df_ezeiza, "t_max"),
    "aeroparque": get_serie(df_aeroparque, "t_max"),
    "observatorio": get_serie(df_bsas_obs, "t_max"),
    "punta_indio": get_serie(df_punta_indio, "t_max")
}

In [ ]:
def dataframe_saltos_diff(
    fechas_salto,
    series_dict,
    nombre_var="t_max"
):

    filas = []

    for fecha in fechas_salto:
        fecha_prev = fecha - pd.Timedelta(days=1)

        fila = {
            "fecha": fecha
        }

        for nombre, serie in series_dict.items():
            val_prev = serie.get(fecha_prev, pd.NA)
            val_act = serie.get(fecha, pd.NA)

            if pd.notna(val_prev) and pd.notna(val_act):
                fila[f"{nombre_var}_diff_{nombre}"] = val_act - val_prev
            else:
                fila[f"{nombre_var}_diff_{nombre}"] = pd.NA

        filas.append(fila)

    return pd.DataFrame(filas)


In [ ]:
dataframe_saltos_diff(
    qc_lp["saltos"]["t_max"].index.tolist(),
    series_tmax,
    nombre_var="t_max"
)

,fecha,t_max_diff_lp,t_max_diff_ezeiza,t_max_diff_aeroparque,t_max_diff_observatorio,t_max_diff_punta_indio
0,2023-08-03,-15.2,-14.4,-15.9,-16.6,-13.9


El salto de 15ºC observado el 2023-08-03 se comprueba en el siguiente [articulo](https://www.0221.com.ar/nota/2023-8-3-7-43-0-volvio-el-invierno-con-sol-y-bajas-temperaturas-asi-va-a-seguir-el-clima-en-la-plata).

In [ ]:
def detectar_outliers_espaciales(
    df,
    estacion_objetivo,
    variable,
    pesos,
    umbral_diferencia=2.5,
    k_iqr=1.5,
    min_estaciones=2
):
    df = df.copy()

    col_obj = f"{variable}_{estacion_objetivo}"

    # -------------------------
    # 1. Outliers locales (IQR)
    # -------------------------
    q1 = df[col_obj].quantile(0.25)
    q3 = df[col_obj].quantile(0.75)
    iqr = q3 - q1

    lim_inf = q1 - k_iqr * iqr
    lim_sup = q3 + k_iqr * iqr

    df["outlier_local"] = (
        (df[col_obj] < lim_inf) | (df[col_obj] > lim_sup)
    )

    resultados = []

    # -------------------------
    # 2. Validación espacial
    # -------------------------
    for fecha, row in df[df["outlier_local"]].iterrows():
        valor_obj = row[col_obj]

        valores_vecinos = []
        pesos_vecinos = []

        for est, w in pesos.items():
            col = f"{variable}_{est}"
            if col in df.columns and not pd.isna(row[col]):
                valores_vecinos.append(row[col])
                pesos_vecinos.append(w)

        if len(valores_vecinos) < min_estaciones:
            resultados.append("indeterminado")
            continue

        ref = np.average(valores_vecinos, weights=pesos_vecinos)

        if abs(valor_obj - ref) <= umbral_diferencia:
            resultados.append("valor_comprobado")
        else:
            resultados.append("valor_a_verificar")

    df_out = df[df["outlier_local"]].copy()
    df_out["clasificacion"] = resultados
    df_out["diferencia_ref"] = (
        df_out[col_obj] -
        df_out.apply(
            lambda r: np.average(
                [r[f"{variable}_{e}"] for e in pesos if f"{variable}_{e}" in df.columns and not pd.isna(r[f"{variable}_{e}"])],
                weights=[pesos[e] for e in pesos if f"{variable}_{e}" in df.columns and not pd.isna(r[f"{variable}_{e}"])]
            ),
            axis=1
        )
    )

    return df_out

In [ ]:
def detectar_outliers_climaticos(df, ventana_dias=15):
    df = df.copy()
    df['dia_del_anio'] = df['fecha'].dt.dayofyear

    outliers = {
        "t_min": [],
        "t_max": [],
        "t_media": [],
        "pp": []
    }

    # Corrección por año bisiesto (día 366)
    df.loc[df['dia_del_anio'] == 366, 'dia_del_anio'] = 365

    for col in ["t_min", "t_max", "t_media"]:
        print(f"\n==============================")
        print(f"Detectando outliers en {col}")
        print(f"==============================\n")

        for i, row in df.iterrows():
            dia = row["dia_del_anio"]
            valor = row[col]

            # seleccionar ventana estacional: día-15 a día+15
            dias_ventana = list(range(dia - ventana_dias, dia + ventana_dias + 1))
            dias_ventana = [d if d >= 1 else d + 365 for d in dias_ventana]
            dias_ventana = [d if d <= 365 else d - 365 for d in dias_ventana]

            sub = df[df["dia_del_anio"].isin(dias_ventana)][col]

            media = sub.mean()
            std = sub.std()

            # regla WMO: ±4 desviaciones estándar
            if std > 0 and abs(valor - media) > 4 * std:
                outliers[col].append({
                    "fecha": row["fecha"],
                    "valor": valor,
                    "media_estacional": media,
                    "std_estacional": std,
                })

    # Precipitación: percentiles extremos
    print("\n==============================")
    print("Detectando outliers en pp (precipitación)")
    print("==============================\n")

    pp_pos = df[df["pp"] > 0]["pp"]  # solo días con lluvia
    p99 = pp_pos.quantile(0.99)
    p995 = pp_pos.quantile(0.995)

    outliers["pp"] = df[df["pp"] > p99][["fecha", "pp"]].to_dict(orient="records")

    print(f"Valores > P99: {len(outliers['pp'])}")

    return outliers

outliers = detectar_outliers_climaticos(df_lp)



Detectando outliers en t_min


Detectando outliers en t_max


Detectando outliers en t_media


Detectando outliers en pp (precipitación)

Valores > P99: 55


Ahora verificamos la existencia de datos faltantes. Para una mayor comprensión de los mismos y como se distribuyen temporalmente, cuantificamos la cantidad de datos faltantes consecutivos y el periodo en el que se encuentran comprendidos

In [ ]:
def rachas_faltantes(df, col):
  faltantes = df[df[col].isna()].copy()
  faltantes = faltantes.sort_values("fecha")
  faltantes["consecutivo"] = (faltantes["fecha"].diff().dt.days == 1)
  faltantes["grupo"] = (~faltantes["consecutivo"]).cumsum()

  rachas = (
      faltantes.groupby("grupo")
      .agg(inicio=("fecha","min"), fin=("fecha","max"), dias=("fecha","count"))
      .reset_index(drop=True)
  )

  print(f"Rachas de días consecutivos de {col} con datos faltantes:")
  print(rachas)

In [ ]:
rachas_faltantes(df_lp, "pp")
print("------------------------------------")
rachas_faltantes(df_lp, "t_min")
print("------------------------------------")
rachas_faltantes(df_lp, "t_max")
print("------------------------------------")
rachas_faltantes(df_lp, "t_media")

Rachas de días consecutivos de pp con datos faltantes:
       inicio        fin  dias
0  1969-05-10 1969-05-10     1
1  1969-12-21 1969-12-21     1
2  1972-01-01 1972-01-31    31
3  1981-11-14 1981-11-14     1
4  1983-03-01 1983-03-31    31
5  1987-01-26 1987-01-26     1
6  1990-09-10 1990-09-10     1
7  1993-08-07 1993-08-07     1
8  1994-04-29 1994-04-29     1
9  2018-12-14 2018-12-14     1
10 2020-01-19 2020-01-19     1
------------------------------------
Rachas de días consecutivos de t_min con datos faltantes:
       inicio        fin  dias
0  1965-02-05 1965-02-05     1
1  1966-05-09 1966-05-09     1
2  1970-06-17 1970-06-17     1
3  1972-01-01 1972-01-31    31
4  1972-02-26 1972-02-26     1
5  1977-02-11 1977-02-11     1
6  1983-03-01 1983-03-31    31
7  1985-08-22 1985-12-31   132
8  1986-11-24 1986-11-24     1
9  1987-01-26 1987-01-26     1
10 1987-02-25 1987-02-25     1
11 1994-09-26 1994-09-27     2
12 1994-11-07 1994-11-07     1
13 1994-11-10 1994-11-10     1
14 1994-11-17

Podemos observar que para todas las variables hay largos periodos de tiempo sin datos disponibles.

### Homogeneidad y completitud de datos

En la presente sección lo que se realizará es:

- Verificación de existencia de puntos de quiebre.
- Generación de modelos de regresión lineal múltiple para completar datos faltantes.

In [ ]:
def get_serie(df, col):
  return pd.Series(df[col].values, index=df['fecha'])

series_por_variable = {
    "t_min": {
        "lp": get_serie(df_lp, "t_min"),
        "ezeiza": get_serie(df_ezeiza, "t_min"),
        "aeroparque": get_serie(df_aeroparque, "t_min"),
        "observatorio": get_serie(df_bsas_obs, "t_min"),
        "punta_indio": get_serie(df_punta_indio, "t_min")
    },
    "t_max": {
        "lp": get_serie(df_lp, "t_max"),
        "ezeiza": get_serie(df_ezeiza, "t_max"),
        "aeroparque": get_serie(df_aeroparque, "t_max"),
        "observatorio": get_serie(df_bsas_obs, "t_max"),
        "punta_indio": get_serie(df_punta_indio, "t_max")
    },
    "t_media": {
        "lp": get_serie(df_lp, "t_media"),
        "ezeiza": get_serie(df_ezeiza, "t_media"),
        "aeroparque": get_serie(df_aeroparque, "t_media"),
        "observatorio": get_serie(df_bsas_obs, "t_media"),
        "punta_indio": get_serie(df_punta_indio, "t_media")
    }
}


In [ ]:
from pyhomogeneity import pettitt_test, snht_test, buishand_u_test

def aplicar_tests_homogeneidad(series_por_estacion):
    """
    Retorna un dict:
    resultados[estacion][variable][test] = {h, cp, p}
    """

    resultados = {}

    for estacion, vars_dict in series_por_estacion.items():
        resultados[estacion] = {}

        for var, serie in vars_dict.items():
            resultados[estacion][var] = {}

            # Pettitt
            res = pettitt_test(serie)
            resultados[estacion][var]["pettitt"] = {
                "h": res.h,
                "cp": res.cp if pd.notna(res.cp) else None,
                "p": res.p
            }

            # SNHT
            res = snht_test(serie)
            resultados[estacion][var]["snht"] = {
                "h": res.h,
                "cp": res.cp if pd.notna(res.cp) else None,
                "p": res.p
            }

            # Buishand
            res = buishand_u_test(serie)
            resultados[estacion][var]["buishand"] = {
                "h": res.h,
                "cp": res.cp if pd.notna(res.cp) else None,
                "p": res.p
            }

    return resultados


In [ ]:
resultados = {'lp': {'t_min': {'pettitt': {'h': np.True_,
    'cp': np.str_('1979-10-09'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1979-10-09'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1979-10-09'),
    'p': np.float64(0.0)}},
  't_max': {'pettitt': {'h': np.True_,
    'cp': np.str_('2003-10-13'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1961-09-23'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('2007-10-15'),
    'p': np.float64(0.0)}},
  't_media': {'pettitt': {'h': np.True_,
    'cp': np.str_('1992-04-05'),
    'p': np.float64(0.0003)},
   'snht': {'h': np.True_, 'cp': np.str_('1967-03-08'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1992-04-05'),
    'p': np.float64(0.0011)}}},
 'aero': {'t_min': {'pettitt': {'h': np.True_,
    'cp': np.str_('1995-10-30'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1961-03-31'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1996-09-29'),
    'p': np.float64(0.0)}},
  't_max': {'pettitt': {'h': np.True_,
    'cp': np.str_('1976-10-28'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1975-10-11'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1976-10-28'),
    'p': np.float64(0.0)}},
  't_media': {'pettitt': {'h': np.True_,
    'cp': np.str_('1995-10-17'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1977-10-16'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1995-10-17'),
    'p': np.float64(0.0)}}},
 'ezeiza': {'t_min': {'pettitt': {'h': np.True_,
    'cp': np.str_('1996-09-30'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1961-03-31'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1996-09-30'),
    'p': np.float64(0.0)}},
  't_max': {'pettitt': {'h': np.True_,
    'cp': np.str_('1988-10-26'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1984-09-29'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1984-10-06'),
    'p': np.float64(0.0)}},
  't_media': {'pettitt': {'h': np.True_,
    'cp': np.str_('1988-11-01'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1995-10-17'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1995-10-17'),
    'p': np.float64(0.0)}}},
 'obs': {'t_min': {'pettitt': {'h': np.True_,
    'cp': np.str_('2013-10-07'),
    'p': np.float64(0.0007)},
   'snht': {'h': np.True_, 'cp': np.str_('1961-03-31'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('2013-10-06'),
    'p': np.float64(0.00195)}},
  't_max': {'pettitt': {'h': np.True_,
    'cp': np.str_('1994-10-31'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('2007-10-15'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1994-10-31'),
    'p': np.float64(0.0)}},
  't_media': {'pettitt': {'h': np.True_,
    'cp': np.str_('1994-10-31'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('1994-10-31'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1994-10-31'),
    'p': np.float64(0.0)}}},
 'pta': {'t_min': {'pettitt': {'h': np.True_,
    'cp': np.str_('2014-10-14'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('2014-10-14'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('2014-10-02'),
    'p': np.float64(0.0)}},
  't_max': {'pettitt': {'h': np.True_,
    'cp': np.str_('1994-10-30'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('2007-10-15'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1994-10-30'),
    'p': np.float64(0.0)}},
  't_media': {'pettitt': {'h': np.True_,
    'cp': np.str_('1988-10-26'),
    'p': np.float64(0.0)},
   'snht': {'h': np.True_, 'cp': np.str_('2007-10-15'), 'p': np.float64(0.0)},
   'buishand': {'h': np.True_,
    'cp': np.str_('1988-10-26'),
    'p': np.float64(0.0)}}}}

In [ ]:
filas = []

for estacion, vars_dict in resultados.items():
    for variable, metodos in vars_dict.items():
        for metodo, fechas in metodos.items():
            filas.append({
                "estacion": estacion,
                "variable": variable,
                "metodo": metodo,
                "fecha_quiebre": fechas["cp"],
                "p_valor": fechas["p"],
                "h": fechas["h"]
            })

quiebres_df = pd.DataFrame(filas)

{'h': np.True_, 'cp': np.str_('1979-10-09'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1979-10-09'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1979-10-09'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('2003-10-13'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1961-09-23'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('2007-10-15'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1992-04-05'), 'p': np.float64(0.0003)}
{'h': np.True_, 'cp': np.str_('1967-03-08'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1992-04-05'), 'p': np.float64(0.0011)}
{'h': np.True_, 'cp': np.str_('1995-10-30'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1961-03-31'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1996-09-29'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1976-10-28'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1975-10-11'), 'p': np.float64(0.0)}
{'h': np.True_, 'cp': np.str_('1976-10-28'), 'p': np.flo

In [ ]:
quiebres_df

,estacion,variable,metodo,fecha_quiebre,p_valor,h
0,lp,t_min,pettitt,1979-10-09,0.00000,True
1,lp,t_min,snht,1979-10-09,0.00000,True
2,lp,t_min,buishand,1979-10-09,0.00000,True
3,lp,t_max,pettitt,2003-10-13,0.00000,True
4,lp,t_max,snht,1961-09-23,0.00000,True
5,lp,t_max,buishand,2007-10-15,0.00000,True
6,lp,t_media,pettitt,1992-04-05,0.00030,True
7,lp,t_media,snht,1967-03-08,0.00000,True
8,lp,t_media,buishand,1992-04-05,0.00110,True
9,aero,t_min,pettitt,1995-10-30,0.00000,True


In [ ]:
quiebres_df.to_csv("quiebres_estaciones.csv", index=False)

In [ ]:
vars_objetivo = ["t_min", "t_max", "t_media"]

lp_df = (
    quiebres_df
    .query("estacion == 'lp' and variable in @vars_objetivo and h == True")
    .copy()
)

lp_df["fecha_quiebre"] = pd.to_datetime(lp_df["fecha_quiebre"])


In [ ]:
lp_consenso = (
    lp_df
    .groupby(["variable", "fecha_quiebre"])
    .agg(
        n_metodos=("metodo", "nunique"),
        p_min=("p_valor", "min")
    )
    .reset_index()
)

In [ ]:
lp_consenso["tipo_quiebre"] = lp_consenso["n_metodos"].apply(
    lambda x: "fuerte" if x >= 2 else "debil"
)

In [ ]:
lp_quiebres_operativos = (
    lp_consenso
    .query("tipo_quiebre == 'fuerte'")
    .sort_values(["variable", "fecha_quiebre"])
)

In [ ]:
def construir_periodos(
    df_quiebres,
    variables,
    fecha_inicio,
    fecha_fin
):
    """
    df_quiebres: DataFrame con columnas [variable, fecha_quiebre]
    variables  : lista de variables objetivo
    """
    periodos = {}

    for var in variables:
        g = df_quiebres[df_quiebres["variable"] == var]

        if g.empty:
            periodos[var] = [
                (fecha_inicio, fecha_fin)
            ]
        else:
            fechas = (
                [fecha_inicio]
                + sorted(g["fecha_quiebre"].tolist())
                + [fecha_fin]
            )

            periodos[var] = [
                (fechas[i], fechas[i + 1])
                for i in range(len(fechas) - 1)
            ]

    return periodos


In [ ]:
fecha_inicio = pd.Timestamp("1961-01-01")
fecha_fin = pd.Timestamp("2024-12-31")

vars_objetivo = ["t_min", "t_max", "t_media"]

periodos_lp = construir_periodos(
    lp_quiebres_operativos,
    variables=vars_objetivo,
    fecha_inicio=fecha_inicio,
    fecha_fin=fecha_fin
)

periodos_lp


{'t_min': [(Timestamp('1961-01-01 00:00:00'),
   Timestamp('1979-10-09 00:00:00')),
  (Timestamp('1979-10-09 00:00:00'), Timestamp('2024-12-31 00:00:00'))],
 't_max': [(Timestamp('1961-01-01 00:00:00'),
   Timestamp('2024-12-31 00:00:00'))],
 't_media': [(Timestamp('1961-01-01 00:00:00'),
   Timestamp('1992-04-05 00:00:00')),
  (Timestamp('1992-04-05 00:00:00'), Timestamp('2024-12-31 00:00:00'))]}

In [ ]:
series_por_variable = {
    "t_min": {
        "objetivo": get_serie(df_lp, "t_min"),
        "predictoras": {
            "ezeiza": get_serie(df_ezeiza, "t_min"),
            "aeroparque": get_serie(df_aeroparque, "t_min"),
            "observatorio": get_serie(df_bsas_obs, "t_min"),
            "punta_indio": get_serie(df_punta_indio, "t_min"),
        }
    },
    "t_max": {
        "objetivo": get_serie(df_lp, "t_max"),
        "predictoras": {
            "ezeiza": get_serie(df_ezeiza, "t_max"),
            "aeroparque": get_serie(df_aeroparque, "t_max"),
            "observatorio": get_serie(df_bsas_obs, "t_max"),
            "punta_indio": get_serie(df_punta_indio, "t_max"),
        }
    },
    "t_media": {
        "objetivo": get_serie(df_lp, "t_media"),
        "predictoras": {
            "ezeiza": get_serie(df_ezeiza, "t_media"),
            "aeroparque": get_serie(df_aeroparque, "t_media"),
            "observatorio": get_serie(df_bsas_obs, "t_media"),
            "punta_indio": get_serie(df_punta_indio, "t_media"),
        }
    }
}


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

def ajustar_regresion_periodo(
    serie_objetivo,
    series_predictoras,
    fecha_ini,
    fecha_fin,
    min_muestras=365
):
    """
    Ajusta una regresión lineal múltiple para un período dado
    """

    # 1. Construir dataframe alineado
    df = pd.DataFrame({
        "y": serie_objetivo.loc[fecha_ini:fecha_fin]
    })

    for nombre, serie in series_predictoras.items():
        df[nombre] = serie.loc[fecha_ini:fecha_fin]

    # 2. Eliminar filas incompletas
    df = df.dropna()

    if len(df) < min_muestras:
        return None

    # 3. Ajustar modelo
    X = df.drop(columns="y")
    y = df["y"]

    modelo = LinearRegression()
    modelo.fit(X, y)

    # 4. Métricas
    y_hat = modelo.predict(X)

    resultado = {
        "modelo": modelo,
        "coeficientes": pd.Series(
            modelo.coef_,
            index=X.columns
        ),
        "intercepto": modelo.intercept_,
        "r2": r2_score(y, y_hat),
        "mae": mean_absolute_error(y, y_hat),
        "n": len(df),
        "df_entrenamiento": df
    }

    return resultado


In [ ]:
def ajustar_modelos_por_variable(
    series_por_variable,
    periodos_por_variable
):
    modelos = {}

    for var, cfg in series_por_variable.items():
        modelos[var] = []

        if var not in periodos_por_variable:
            continue

        for (ini, fin) in periodos_por_variable[var]:
            res = ajustar_regresion_periodo(
                cfg["objetivo"],
                cfg["predictoras"],
                ini,
                fin
            )

            if res is not None:
                res["periodo"] = (ini, fin)
                modelos[var].append(res)

    return modelos


In [ ]:
modelos_lp = ajustar_modelos_por_variable(
    series_por_variable,
    periodos_lp
)

In [ ]:
modelos_lp

{'t_min': [{'modelo': LinearRegression(),
   'coeficientes': ezeiza          0.676083
   aeroparque     -0.156180
   observatorio    0.103240
   punta_indio     0.367955
   dtype: float64,
   'intercepto': np.float64(-1.050948749128036),
   'r2': 0.9287985219614026,
   'mae': 1.228452101730917,
   'n': 6575,
   'df_entrenamiento':                y  ezeiza  aeroparque  observatorio  punta_indio
   fecha                                                          
   1961-01-01  18.8    23.6        24.6          23.0         18.1
   1961-01-02  17.0    18.7        19.8          19.6         18.1
   1961-01-03  14.3    15.4        20.0          19.0         21.1
   1961-01-04  15.3    17.0        20.0          18.6         15.1
   1961-01-05  18.3    18.4        20.6          20.6         18.1
   ...          ...     ...         ...           ...          ...
   1979-10-05  11.8    11.0        12.2          11.8         11.8
   1979-10-06   8.0     7.8        11.3          10.3         10.8


In [ ]:
def imputar_lp_con_modelos(
    serie_lp,
    modelos_variable,
    series_predictoras
):

    serie_imputada = serie_lp.copy()
    log = []

    for bloque in modelos_variable:
        modelo = bloque["modelo"]
        fecha_ini, fecha_fin = bloque["periodo"]

        pred_names = bloque["coeficientes"].index.tolist()

        fechas = serie_imputada.loc[fecha_ini:fecha_fin].index

        for fecha in fechas:
            if not pd.isna(serie_imputada.loc[fecha]):
                continue

            X = []
            valido = True

            for est in pred_names:
                val = series_predictoras[est].get(fecha, pd.NA)
                if pd.isna(val):
                    valido = False
                    break
                X.append(val)

            if not valido:
                continue

            X_df = pd.DataFrame([X], columns=pred_names)
            y_pred = modelo.predict(X_df)[0]
            serie_imputada.loc[fecha] = y_pred

            log.append({
                "fecha": fecha,
                "valor_imputado": y_pred,
                "periodo_ini": fecha_ini,
                "periodo_fin": fecha_fin
            })

    return serie_imputada, pd.DataFrame(log)


In [ ]:
serie_tmin_lp_imp, log_tmin = imputar_lp_con_modelos(
    serie_lp=series_por_variable['t_min']['objetivo'],
    modelos_variable=modelos_lp["t_min"],
    series_predictoras=series_por_variable["t_min"]["predictoras"]
)

serie_tmax_lp_imp, log_tmax = imputar_lp_con_modelos(
    serie_lp=series_por_variable['t_max']['objetivo'],
    modelos_variable=modelos_lp["t_max"],
    series_predictoras=series_por_variable["t_max"]["predictoras"]
)

serie_tmedia_lp_imp, log_tmedia = imputar_lp_con_modelos(
    serie_lp=series_por_variable['t_media']['objetivo'],
    modelos_variable=modelos_lp["t_media"],
    series_predictoras=series_por_variable["t_media"]["predictoras"]
)

In [ ]:
df_lp_imputada = pd.concat([serie_tmin_lp_imp, serie_tmax_lp_imp, serie_tmedia_lp_imp, get_serie(df_lp, "pp")], axis=1)
df_lp_imputada.columns = ["t_min", "t_max", "t_media", "pp"]

df_lp_imputada = df_lp_imputada.round(1).reset_index()
df_lp_imputada

,fecha,t_min,t_max,t_media,pp
0,1961-01-01,18.8,34.7,29.0,0.0
1,1961-01-02,17.0,30.9,23.5,0.0
2,1961-01-03,14.3,29.1,23.0,0.0
3,1961-01-04,15.3,29.4,23.9,0.0
4,1961-01-05,18.3,25.0,22.0,0.0
...,...,...,...,...,...
23371,2024-12-27,12.3,27.5,19.6,0.0
23372,2024-12-28,12.1,27.3,18.9,0.0
23373,2024-12-29,11.5,27.0,19.8,0.0
23374,2024-12-30,14.0,26.8,20.7,0.0


In [ ]:
df_lp_imputada.to_csv(f"{PATH_OUTPUT}/lp_imputada.csv", index=False)